In [1]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages, MessagesState
from typing import Annotated
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect("memory.db", check_same_thread=False)

# llm.invoke([{
#     "role": "user",
#     "content": "hi!"
# }])

In [2]:
# class State(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]

class State(MessagesState):
    pass

graph_builder = StateGraph(State)

In [3]:
@tool
def get_weather(city: str) -> str:
    """Get the weather in a given city"""
    return f"The weather in {city} is sunny"


llm_with_tools = llm.bind_tools(tools=[get_weather])


def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {
        "messages": [response]
    }


In [4]:
tool_node = ToolNode(
    tools=[
        get_weather,
    ]
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
# graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn)
)

In [5]:
graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "안녕 난 성식이야.  부산의 오늘 날씨를 알려줘"
            }
        ]
    },
    config={
        "configurable":{
            "thread_id": "1"
        }

    }
)

{'messages': [HumanMessage(content='안녕 난 성식이야.  부산의 오늘 날씨를 알려줘', additional_kwargs={}, response_metadata={}, id='4fa6cebd-4fc4-4cea-a542-bf1f4d3ff18e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lar9Ggq2QW6apALtNJJ5Qchq', 'function': {'arguments': '{"city":"부산"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 62, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CPpvG9arMABGzY8GIuCApM8EI3Hzs', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f65f8206-a0cd-426e-a215-c8be2dd2c4d7-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '부산'}, 'id': 'call_lar9Ggq2Q

In [ ]:
graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "내가 물어본곳이 어디었지?"
            }
        ]
    },
    config={
        "configurable":{
            "thread_id": "1"
        }

    }
)

In [6]:
print(graph.get_graph().draw_ascii())

        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +---------+         
          | chatbot |         
          +---------+         
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 
